# Creating IIIF Manifests

|Category(ies)         |References            |Input Type            |Workflows             |Next Steps            |
|----------------------|----------------------|----------------------|----------------------|----------------------|
|conversion|[IIIF website](https://iiif.io/), [iiif-prezi3 package](https://github.com/iiif-prezi/iiif-prezi3)|multiple|||

In this notebook we will create [IIIF](https://iiif.io/) Manifests from different kinds of media. A IIIF Manifest is a light weight json file that allows us to point to media that is hosted online (or several medias in one Manifest), and also define things like metadata and annotations. It follows a standardized structure that is defined by the [IIIF Presentation API](https://iiif.io/api/presentation/3.0/).

## 1. Setup
First, let's import all of our dependencies.

In [ ]:
# Installs
import sys
!echo "Purging pip environment and installing packages..."
!{sys.executable} -m pip cache purge > /dev/null 2>&1 # Purge pip's cache so that everything installs correctly.
!{sys.executable} -m pip uninstall -y jlu > /dev/null 2>&1 # Uninstall jlu if it was previously installed.
!{sys.executable} -m pip install -q git+https://github.com/arvest-data-in-context/jlu > /dev/null 2>&1 # Install the jlu package.
!conda install -y -c conda-forge ffmpeg > /dev/null 2>&1 # Install ffmpeg for file conversion.
!{sys.executable} -m pip install -q git+https://github.com/iiif-prezi/iiif-prezi3.git # Install the iiif-prezi3 package.

# Imports
print("Importing packages...")
from jlu import files
import iiif_prezi3
import uuid
import os
import json
print("Ready!")

## 2. Prepare your media
In order to work, IIIF needs the URL of an accesible media file on the internet. There are plenty of services that let you store files and render them accessible, some of them are free, some you will have to pay for. Here are a few:
- [Internet archive](https://archive.org/): you can store any type of file on internet archive.
- [Github](https://github.com/): if you have a github account, you can upload small files to a repo and access them with a direct url.
- [Nakala](https://www.nakala.fr/): run by Huma-Num, Nakala is a service that allows you to store data for digital humanities projects.
- [File Browser](https://filebrowser.org/): file browser is an open source file hosting system you can set up on your own server.
- [Arvest](https://arvest.app): Arvest is a digital humanities tool specifically designed for IIIF which also lets you store media.
- [Imgur](https://imgur.com/): a free image storing service.
- [Dropbox](https://www.dropbox.com/): you will need to change the prefix of the link link from `www.dropbox.com` to `dl.dropboxusercontent.com`.
- [Amazon S3 (Simple Storage Service](https://aws.amazon.com/fr/s3/): a paid service provided by amazon.
- [pCloud](https://www.pcloud.com/fr/eu): a paid storage solution.
- [Wikimedia Commons](https://commons.wikimedia.org/wiki/Accueil): a free storage solution provided by the Wikimedia Foundation.

You will know that a file is directly available if you go to the link in an internet browser and nothing but the media file's contents appears. For the purposes of this module, we shall use some content that we know is available at the following adresses (follow the links to see them), however you can always replace these file names with your own. 

In [ ]:
media_urls = {
    "image_1" : "https://files.tetras-libre.fr/media/sceno_avignon.jpg",
    "image_2" : "https://files.tetras-libre.fr/media/walden_nouvel.JPG",
    "audio_1" : "https://files.tetras-libre.fr/media/test_markeas.wav",
    "audio_2" : "https://files.tetras-libre.fr/media/speech_recognition_test_corpus/en/clean-support-call.wav",
    "video_1" : "https://files.tetras-libre.fr/media/jf_peyret_re_walden.mp4",
    "video_2" : "https://files.tetras-libre.fr/media/plozevet/fonds_gessain/132.mp4"
}

## 3. Create a basic Manifest

Let's create our first Manifest. Remeber, a Manifest is just a json file with a certain structure, we could do everything by hand, but we'll use the [iiif-prezi3](https://github.com/iiif-prezi/iiif-prezi3) package to make things easier. 

We can create an instance of the Manifest class to represent our IIIF Manifest.

In [ ]:
filename = f"{str(uuid.uuid4())}.json" # First, let's set a filename four our Manifest (we just set a random one).

m = iiif_prezi3.Manifest(
    # The id is the URL to the Manifest when it will be uploaded online:
    id = f"https://files.tetras-libre.fr/manifests/{filename}",
    # We must also give our Manifest a label using the w3c annotations model):
    label = {"en" : ["My first Manifest"]}
)

# Now we can print the Manifest in json form to see its structure:
print(m.json(indent=2))

Great! Now if we want, we can turn this into a json file and save the Manifest onto our machine so that we can upload it later.

In [ ]:
# We will save the json to the "output" folder at the root of the ml-notebooks repo:
OUTPUT_FOLDER = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..", "output"))

# Creating a jlu File object for clean file handling:
file = files.File(path = os.path.join(OUTPUT_FOLDER, filename), content = m.json())
file.write()

print(f"Created {file.path}!")

## 4. Creating Image Manifests

Now that we know how to create a basic manifest, let's create one that points to an image, and also gives it some metadata and annotations.

To do this, we'll need to get some basic information about our image (namely, its dimensions). We'll first need to be able to access our image. We could do this using an API if available, but we'll just download it to our machine to make things simpler.

In [ ]:
# We will download the file to the "sources" folder at the root of the ml-notbooks repo:
SOURCE_FOLDER = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..", "sources"))

# Now let's download the image files using jlu's download function:
downloaded_files = files.download([media_urls["image_1"], media_urls["image_2"]], SOURCE_FOLDER)

# We have created two new jlu File objects that represent the downloaded files which are here:
print(downloaded_files[0].path)
print(downloaded_files[1].path)

An image file's dimensions are part of the file's basic properties. We can use jlu's `get_file_properties()` method to retrieve this data (you can also make jlu run this automatically on File creating using `get_file_properties` kwarg to True).

In [ ]:
for file in downloaded_files:
    file.get_file_properties()
    print(file.file_properties, "\n")

### Creating the Manifest

Now that we have all the information we need, we can create the actual Manifest using the same ethod seen above. We shall add to the Manifest a Canvas (the blank canvas that we'll paint our content on), to the Canvas we'll add an Annotation Page (the container of our media), and to the Annotation Page, we'll add an Annotation (the actual media content). The terminology can be a bit confusing - this is because we're using the [w3c web annotations](https://www.w3.org/TR/annotation-model/) terms - but you'll soon get used to it!

You can learn more about the structure of the Presentation API modele [here](https://iiif.io/api/presentation/3.0/#21-defined-types). Here is a breakdown of the model in image form:

<img src="https://iiif.io/api/assets/images/data-model.png" alt="IIIF Presentation API structure" style="width:400px; padding-left:100px;"/>

Let's put this into action now!

In [ ]:
# Retrieve the filename without extension for naming purposes:
just_filename = os.path.splitext(os.path.basename(downloaded_files[0].filename))[0]

# The main Manifest object:
m = iiif_prezi3.Manifest(
    id = f"https://files.tetras-libre.fr/manifests/{just_filename}.json",
    label = {"en" : [just_filename]}
)

# Add a Canvas to the Manifest:
c = iiif_prezi3.Canvas(
    # Each Canvas has an id (the last number will increment if there is more than one):
    id = f"https://files.tetras-libre.fr/manifests/{just_filename}/canvas/1",
    # Each Canvas must also have a label:
    label = {"en" : [downloaded_files[0].filename]},
    # Let's give our Canvas a width and a height equal to that of our image:
    width = downloaded_files[0].file_properties["width"],
    height = downloaded_files[0].file_properties["height"]
)
m.items = [c] # Add the Canvas to the Manifest's "items"

# Add an Annotation Page to the Canvas:
ap = iiif_prezi3.AnnotationPage(
    # The page's if first points to the Canvas, then this page:
    id = f"https://files.tetras-libre.fr/manifests/{just_filename}/canvas/1/page/1"
)
c.items = [ap] # Add the Annotation Page to the Canvas's "items"

# Finally, add an Annotation to the Annotation Page (this adds our actual content):
a = iiif_prezi3.Annotation(
    # For its id, we point to the Annotation Page it is on:
    id = f"https://files.tetras-libre.fr/manifests/{just_filename}/canvas/1/page/1/1",
    # We can also give it a motivation following the w3c terms:
    motivation = "painting",
    # We also define a target, telling IIIF which Canvas to paint it on, and where:
    target = f"{c.id}#xywh=0,0,{c.width},{c.height}",
    # Finaly, the body is where we shall give our media file info:
    body = {
        "id" : media_urls["image_1"], # We give the url to where the media is found online
        "type" : downloaded_files[0].mime[0].capitalize(), # The type of file
        "format" : f"{downloaded_files[0].mime[0]}/{downloaded_files[0].mime[1]}", # File format
        "width" : downloaded_files[0].file_properties["width"], # The image width
        "height" : downloaded_files[0].file_properties["height"] # And height
    }
)
ap.items = [a] # Add the Annotation to the Annotation Page's "items"

# Now we can print out the Manifest, or save it like before if need be.
print(m.json(indent=2))

### Metadata and other information

We now have a fully IIIF-compliant Manifest that displays our image. However, we may want to pack up some other information with it, for example metadata, credits, a logo, required statement etc. We can do this following w3c specifications as follows:

In [ ]:
# License:
m.rights = "https://creativecommons.org/licenses/by-nc-nd/4.0/"

# Provider:
m.provider = [{
    "id" : "https://www.univ-rennes2.fr/",
    "type" : "Agent",
    "label" : {"en" : ["Rennes 2 University"]},
    "homepage" : {"id" : "https://www.univ-rennes2.fr/", "type" : "Text", "format": "text/html", "label" : {"en" : ["Rennes 2 University website"]}}
}]

# Required Statement:
m.requiredStatement = {
    "label" : {"en" : ["Attribution"]},
    "value" : {"en" : ["Rennes 2 University"]},
}

# Metadata:
m.metadata = [
    {"label" : {"en" : ["Title"]}, "value" : {"en" : ["My first Manifest"]}},
    {"label" : {"en" : ["Date"]}, "value" : {"en" : ["2024"]}}
    # You can go on and add any number of metadata you wish
]

# See the updated Manifest:
print(m.json(indent=2))

### Logo and thumbnail

We can also give our Canvas a thumbnail which acts as a preview of what is found on the Canvas. We'll just use the same image for now, but note that it is best to use a smaller image as a thumbnail.

Some IIIF viewers can also accept of Manifest logo, but this is not officially in the IIIF specs. This means that we'll have to add it to the json data once it is no longer a `iiif_prezi3.Manifest()` object.

In [ ]:
# Using the same image as the Annotation body defined above:
c.thumbnail = [a.body]

# To add a logo, we have to convert the manifest to a dict, and then add it to that:
manifest_as_dict = json.loads(m.json())
manifest_as_dict["logo"] = {
    "id": "https://univ-droit.fr/images/structures/universites/356/0350937d.jpg",
    "type": "Image",
    "format": "image/png",
    "height": 833,
    "width": 413
}

# See the updated Manifest (which is now dict data):
print(manifest_as_dict)

### Annotations

Finally, we can add some annotations to our Manifest. Annotations are thought of as complementary information that, unlike metadata, target a specific part of the resource.

Annotations are added to the Canvas not in its `"items"` but in its `"annotations"`. We will add to this a new Annotation Page, and then an Annotation for each annotation we wish to add. Annotations are where IIIF specs and usage can begin to misalign, and things get a bit murky with different viewers treating different stuctures differently. The writer of `iiif_prezi3` has made the choice to not allow us to add annotations of non fully-compliant formats. If we want to create annotations that will be viewable in a viewer like [Arvest](https://arvest.app), we shall have to proceed like logos, and add the body of an annotation after having transformed the `iiif_prezi3.Manifest()` object into a dict.

In [ ]:
# Create a new Annotation Page:
new_ap = iiif_prezi3.AnnotationPage(
    # For the id, as before we point to the Canvas, but change page to annotation:
    id = f"https://files.tetras-libre.fr/manifests/{just_filename}/canvas/1/annotation/1"
)
c.annotations = [new_ap] # Add the Annotation Page to the Canvas's "annotations"

# Create an annotation:
new_a = iiif_prezi3.Annotation(
    id = f"https://files.tetras-libre.fr/manifests/{just_filename}/canvas/1/annotation/1/1",
    # Note that the motivation changes from painting to commenting:
    motivation = "commenting",
    # Target a region on the Canvas to annotate, here we put it a 100, 100 with a size of 50:
    target = f"{c.id}#xywh=100,100,50,50"
)
new_ap.items = [new_a] # Add the Annotation to the Annotation Page's "items"

# Now we can transform the Manifest object into a dict:
manifest_as_dict = json.loads(m.json())

# And add the annotation body to this:
manifest_as_dict["items"][0]["annotations"][0]["items"][0]["body"] = {
    "id" : m.id,
    "value" : "My textual annotation",
    "type" : "TextualBody",
    "format" : "text/html"
}

# See the updated Manifest:
print(manifest_as_dict)

## 5. Creating video Manifests